# Final Project

## Predict whether a mammogram mass is benign or malignant



This data contains 961 instances of masses detected in mammograms, and contains the following attributes:


   1. BI-RADS assessment: 1 to 5 (ordinal)  
   2. Age: patient's age in years (integer)
   3. Shape: mass shape: round=1 oval=2 lobular=3 irregular=4 (nominal)
   4. Margin: mass margin: circumscribed=1 microlobulated=2 obscured=3 ill-defined=4 spiculated=5 (nominal)
   5. Density: mass density high=1 iso=2 low=3 fat-containing=4 (ordinal)
   6. Severity: benign=0 or malignant=1 (binominal)
   
BI-RADS is an assesment of how confident the severity classification is; it is not a "predictive" attribute and so we will discard it. The age, shape, margin, and density attributes are the features that we will build our model with, and "severity" is the classification we will attempt to predict based on those attributes.

Although "shape" and "margin" are nominal data types, which sklearn typically doesn't deal with well, they are close enough to ordinal that we shouldn't just discard them. The "shape" for example is ordered increasingly from round to irregular.

A lot of unnecessary anguish and surgery arises from false positives arising from mammogram results. If we can build a better way to interpret them through supervised machine learning, it could improve a lot of lives.

## Goals

Apply several different supervised machine learning techniques to this data set, and see which one yields the highest accuracy as measured with K-Fold cross validation (K=10). Apply:

* Decision tree
* Random forest
* KNN
* Naive Bayes
* SVM
* Logistic Regression
* And, as a bonus challenge, a neural network using Keras.

The data needs to be cleaned; many rows contain missing data, and there may be erroneous data identifiable as outliers as well.




## Let's begin: preparing the data

Start by importing the mammographic_masses.data.txt file into a Pandas dataframe .

In [2]:
import pandas as pd
col_names = ['BI-RADS','Age','Shape','Margin','Density','Severity']
data = pd.read_csv("mammographic_masses.data.txt",names=col_names)
data.head(10)


,BI-RADS,Age,Shape,Margin,Density,Severity
0,5,67,3,5,3,1
1,4,43,1,1,?,1
2,5,58,4,5,3,1
3,4,28,1,1,3,0
4,5,74,1,5,?,1
5,4,65,1,?,3,0
6,4,70,?,?,3,0
7,5,42,1,?,3,0
8,5,57,1,5,3,1
9,5,60,?,5,1,1


In [3]:
import numpy as np
df = data.replace('?',np.nan)
df.head(10)

,BI-RADS,Age,Shape,Margin,Density,Severity
0,5,67,3,5,3,1
1,4,43,1,1,NaN,1
2,5,58,4,5,3,1
3,4,28,1,1,3,0
4,5,74,1,5,NaN,1
5,4,65,1,NaN,3,0
6,4,70,NaN,NaN,3,0
7,5,42,1,NaN,3,0
8,5,57,1,5,3,1
9,5,60,NaN,5,1,1


## Analysing the data

In [4]:
print(df.shape)
print(df.size)
print(df.describe())

(961, 6)
5766
         Severity
count  961.000000
mean     0.463059
std      0.498893
min      0.000000
25%      0.000000
50%      0.000000
75%      1.000000
max      1.000000


In [5]:
import seaborn as sns
sns.countplot(df['Severity'],label='count')
B,M = df['Severity'].value_counts()
print("Benign",B)
print("Malignant",M)

Benign 516
Malignant 445


In [6]:
dataset = df.dropna()
print(dataset.shape)
dataset.head()


(830, 6)


,BI-RADS,Age,Shape,Margin,Density,Severity
0,5,67,3,5,3,1
2,5,58,4,5,3,1
3,4,28,1,1,3,0
8,5,57,1,5,3,1
10,5,76,1,4,3,1


### Converting the data to Numpy Array

In [7]:
x = dataset.iloc[:,1:5]
X = np.array(x)
print(X.shape)
X

(830, 4)


array([['67', '3', '5', '3'],
       ['58', '4', '5', '3'],
       ['28', '1', '1', '3'],
       ...,
       ['64', '4', '5', '3'],
       ['66', '4', '5', '3'],
       ['62', '3', '3', '3']], dtype=object)

In [8]:
y = dataset.iloc[:,5]
Y = np.array(y)  
print(Y.shape)
#print(Y)

(830,)


In [9]:
label_names= np.array(col_names[1:])
label_names

array(['Age', 'Shape', 'Margin', 'Density', 'Severity'], dtype='<U8')

### Some of the models require the input data to be normalized, normalizing the attribute data. 

In [10]:
from sklearn.preprocessing import StandardScaler
sc = StandardScaler()
x_norm = sc.fit_transform(X)
x_norm

C:\ProgramData\Anaconda3\lib\site-packages\sklearn\utils\validation.py:595: DataConversionWarning: Data with input dtype object was converted to float64 by StandardScaler.
  warnings.warn(msg, DataConversionWarning)
C:\ProgramData\Anaconda3\lib\site-packages\sklearn\utils\validation.py:595: DataConversionWarning: Data with input dtype object was converted to float64 by StandardScaler.
  warnings.warn(msg, DataConversionWarning)


array([[ 0.7650629 ,  0.17563638,  1.39618483,  0.24046607],
       [ 0.15127063,  0.98104077,  1.39618483,  0.24046607],
       [-1.89470363, -1.43517241, -1.157718  ,  0.24046607],
       ...,
       [ 0.56046548,  0.98104077,  1.39618483,  0.24046607],
       [ 0.69686376,  0.98104077,  1.39618483,  0.24046607],
       [ 0.42406719,  0.17563638,  0.11923341,  0.24046607]])

## Decision Trees

Before moving to K-Fold cross validation and random forests, started creating a single train/test split of our data. Setting aside 75% for training, and 25% for testing.

In [11]:
from sklearn.model_selection import train_test_split
x_train,x_test,y_train,y_test = train_test_split(X,Y,test_size=0.25)
print(x_train.shape)
print(y_train.shape)
print(x_test.shape)
print(y_test.shape)
print("train data",round(622/(622+208) ,2),"%")
print("test data", round(208/ (622+208),2),"%")      

(622, 4)
(622,)
(208, 4)
(208,)
train data 0.75 %
test data 0.25 %


 Now create a DecisionTreeClassifier and fit it to your training data.

In [12]:
from sklearn.tree import DecisionTreeClassifier
model = DecisionTreeClassifier(random_state=1)
model.fit(x_train,y_train)

DecisionTreeClassifier(class_weight=None, criterion='gini', max_depth=None,
            max_features=None, max_leaf_nodes=None,
            min_impurity_decrease=0.0, min_impurity_split=None,
            min_samples_leaf=1, min_samples_split=2,
            min_weight_fraction_leaf=0.0, presort=False, random_state=1,
            splitter='best')

Measure the accuracy of the resulting decision tree model using test data.

In [14]:
from sklearn.metrics import *
prediction = model.predict(x_test)
print("Accuracy of dct is",accuracy_score(prediction,y_test))
print("Error", mean_absolute_error(prediction,y_test))

Accuracy of dct is 0.7403846153846154
Error 0.25961538461538464


In [15]:
from sklearn.model_selection import cross_val_score
cv = cross_val_score(model,x_train,y_train,cv=10)
cv.mean()

0.7398533571722361

In [16]:
from sklearn.ensemble import RandomForestClassifier
rfc = RandomForestClassifier(n_estimators=10,random_state=2)
rfc.fit(x_train,y_train)
prediction = rfc.predict(x_test)
print("Accuracy of rfc is",accuracy_score(prediction,y_test))
print("Error", mean_absolute_error(prediction,y_test))
cv = cross_val_score(rfc,x_train,y_train,cv=10)
cv.mean()

Accuracy of rfc is 0.7451923076923077
Error 0.2548076923076923


0.75915615320692

## SVM

In [18]:
from sklearn import svm
svm = svm.SVC(kernel='linear')
svm.fit(x_train,y_train)
prediction = svm.predict(x_test)
print("Accuracy of svm is",accuracy_score(prediction,y_test))
print("Error", mean_absolute_error(prediction,y_test))


Accuracy of svm is 0.8221153846153846
Error 0.1778846153846154


## KNN 

In [19]:
from sklearn.neighbors import KNeighborsClassifier
model = KNeighborsClassifier(n_neighbors=5)
model.fit(x_train,y_train)
prediction = model.predict(x_test)
print("Accuracy of svm is",accuracy_score(prediction,y_test))
print("Error", mean_absolute_error(prediction,y_test))

cv = cross_val_score(model,x_train,y_train,cv=10)
cv.mean()


Accuracy of svm is 0.7596153846153846
Error 0.2403846153846154


0.7943046846801474

Choosing K is tricky, so  try different values of K. A loop to run KNN with K values ranging from 1 to 50 and see if K makes a substantial difference.

In [20]:
for i in range(1,50):
    model = KNeighborsClassifier(n_neighbors=i)
    cv = cross_val_score(model,x_train,y_train,cv=10)
    print(i,cv.mean())

1 0.7141211922808962
2 0.7139898264964367
3 0.7667556428529458
4 0.7669885757934409
5 0.7943046846801474
6 0.7862393291531313
7 0.7927685863698556
8 0.7782524573375974
9 0.7847561129158167
10 0.7846784686023184
11 0.7846537063617973
12 0.7701879412085652
13 0.7830408031359909
14 0.7717232001208734
15 0.7846545457597811
16 0.7862145669126103
17 0.7942799224396262
18 0.7878018684999119
19 0.7814014588736957
20 0.7700334919795523
21 0.7845496210118104
22 0.7781236097470894
23 0.7829887604609974
24 0.7780724064700797
25 0.7782276950970763
26 0.7717232001208734
27 0.7684709526327718
28 0.7716455558073749
29 0.770084695256562
30 0.7699277278335978
31 0.7683924689212897
32 0.7699797705085912
33 0.7699797705085912
34 0.7747937179454895
35 0.7731799753216992
36 0.7635017165688768
37 0.769901286797109
38 0.769926888435614
39 0.766726683622506
40 0.7618623723065817
41 0.7602494690807753
42 0.7570492642676672
43 0.76027507071928
44 0.7602494690807753
45 0.7618879739450864
46 0.7570748659061721
47 

## Naive Bayes


In [21]:
from sklearn.naive_bayes import MultinomialNB
nb = MultinomialNB()
nb.fit(x_train,y_train)
prediction = nb.predict(x_test)
print("Accuracy of svm is",accuracy_score(prediction,y_test))
print("Error", mean_absolute_error(prediction,y_test))

cv = cross_val_score(model,x_train,y_train,cv=10)
cv.mean()

Accuracy of svm is 0.7548076923076923
Error 0.24519230769230768


0.7603535544307622


## Revisiting SVM


In [22]:
from sklearn import svm
svm = svm.SVC(kernel='rbf',C=1)
svm.fit(x_train,y_train)
prediction = svm.predict(x_test)
print("Accuracy of svm is",accuracy_score(prediction,y_test))
print("Error", mean_absolute_error(prediction,y_test))

Accuracy of svm is 0.7980769230769231
Error 0.20192307692307693


C:\ProgramData\Anaconda3\lib\site-packages\sklearn\svm\base.py:196: FutureWarning: The default value of gamma will change from 'auto' to 'scale' in version 0.22 to account better for unscaled features. Set gamma explicitly to 'auto' or 'scale' to avoid this warning.
  "avoid this warning.", FutureWarning)


In [23]:
from sklearn import svm
svm = svm.SVC(kernel='sigmoid',C=1)
svm.fit(x_train,y_train)
prediction = svm.predict(x_test)
print("Accuracy of svm is",accuracy_score(prediction,y_test))
print("Error", mean_absolute_error(prediction,y_test))

Accuracy of svm is 0.47115384615384615
Error 0.5288461538461539


C:\ProgramData\Anaconda3\lib\site-packages\sklearn\svm\base.py:196: FutureWarning: The default value of gamma will change from 'auto' to 'scale' in version 0.22 to account better for unscaled features. Set gamma explicitly to 'auto' or 'scale' to avoid this warning.
  "avoid this warning.", FutureWarning)


In [ ]:
from sklearn import svm
svm = svm.SVC(kernel='poly',C=1)
svm.fit(x_train,y_train)
prediction = svm.predict(x_test)
print("Accuracy of svm is",accuracy_score(prediction,y_test))
print("Error", mean_absolute_error(prediction,y_test))

## Logistic Regression

In [26]:
from sklearn.linear_model import LogisticRegression
lr = LogisticRegression()
lr.fit(x_train,y_train)
prediction = lr.predict(x_test)
print("Accuracy of svm is",accuracy_score(prediction,y_test))
print("Error", mean_absolute_error(prediction,y_test))

cv = cross_val_score(lr,x_train,y_train,cv=10)
cv.mean()

C:\ProgramData\Anaconda3\lib\site-packages\sklearn\linear_model\logistic.py:433: FutureWarning: Default solver will be changed to 'lbfgs' in 0.22. Specify a solver to silence this warning.
  FutureWarning)


Accuracy of svm is 0.7980769230769231
Error 0.20192307692307693


C:\ProgramData\Anaconda3\lib\site-packages\sklearn\linear_model\logistic.py:433: FutureWarning: Default solver will be changed to 'lbfgs' in 0.22. Specify a solver to silence this warning.
  FutureWarning)
C:\ProgramData\Anaconda3\lib\site-packages\sklearn\linear_model\logistic.py:433: FutureWarning: Default solver will be changed to 'lbfgs' in 0.22. Specify a solver to silence this warning.
  FutureWarning)
C:\ProgramData\Anaconda3\lib\site-packages\sklearn\linear_model\logistic.py:433: FutureWarning: Default solver will be changed to 'lbfgs' in 0.22. Specify a solver to silence this warning.
  FutureWarning)
C:\ProgramData\Anaconda3\lib\site-packages\sklearn\linear_model\logistic.py:433: FutureWarning: Default solver will be changed to 'lbfgs' in 0.22. Specify a solver to silence this warning.
  FutureWarning)
C:\ProgramData\Anaconda3\lib\site-packages\sklearn\linear_model\logistic.py:433: FutureWarning: Default solver will be changed to 'lbfgs' in 0.22. Specify a solver to silence t

0.797556932168249


##  ANN Neural Networks


In [27]:
import tensorflow
import keras
from keras.models import Sequential
from keras.layers import Dense


Using TensorFlow backend.


In [28]:
classifier = Sequential()
classifier.add(Dense(output_dim =8 ,init = 'normal', activation = 'relu', input_dim =4 ))
classifier.add(Dense(output_dim =4 ,init = 'normal', activation = 'relu' ))
classifier.add(Dense(output_dim =1 ,init = 'normal', activation = 'sigmoid'))
classifier.compile(optimizer = 'Adam',loss='binary_crossentropy',metrics=['accuracy'])
 

C:\ProgramData\Anaconda3\lib\site-packages\ipykernel_launcher.py:2: UserWarning: Update your `Dense` call to the Keras 2 API: `Dense(activation="relu", input_dim=4, units=8, kernel_initializer="normal")`
  


Instructions for updating:
Use tf.where in 2.0, which has the same broadcast rule as np.where


C:\ProgramData\Anaconda3\lib\site-packages\ipykernel_launcher.py:3: UserWarning: Update your `Dense` call to the Keras 2 API: `Dense(activation="relu", units=4, kernel_initializer="normal")`
  This is separate from the ipykernel package so we can avoid doing imports until
C:\ProgramData\Anaconda3\lib\site-packages\ipykernel_launcher.py:4: UserWarning: Update your `Dense` call to the Keras 2 API: `Dense(activation="sigmoid", units=1, kernel_initializer="normal")`
  after removing the cwd from sys.path.


In [29]:
classifier.fit(x_train,y_train,batch_size=60,nb_epoch=200)

C:\ProgramData\Anaconda3\lib\site-packages\ipykernel_launcher.py:1: UserWarning: The `nb_epoch` argument in `fit` has been renamed `epochs`.
  """Entry point for launching an IPython kernel.



Epoch 1/200
622/622 [==============================] - 2s 3ms/step - loss: 0.6925 - accuracy: 0.4711
Epoch 2/200
622/622 [==============================] - 0s 56us/step - loss: 0.6917 - accuracy: 0.4711
Epoch 3/200
622/622 [==============================] - 0s 48us/step - loss: 0.6907 - accuracy: 0.4775
Epoch 4/200
622/622 [==============================] - 0s 51us/step - loss: 0.6896 - accuracy: 0.4775
Epoch 5/200
622/622 [==============================] - 0s 51us/step - loss: 0.6883 - accuracy: 0.4727
Epoch 6/200
622/622 [==============================] - 0s 47us/step - loss: 0.6867 - accuracy: 0.4759
Epoch 7/200
622/622 [==============================] - 0s 44us/step - loss: 0.6858 - accuracy: 0.4823
Epoch 8/200
622/622 [==============================] - 0s 40us/step - loss: 0.6842 - accuracy: 0.4952
Epoch 9/200
622/622 [==============================] - 0s 39us/step - loss: 0.6826 - accuracy: 0.5193
Epoch 10/200
622/622 [==============================] - 0s 60us/step - loss: 0.680

622/622 [==============================] - 0s 67us/step - loss: 0.5118 - accuracy: 0.7894
Epoch 79/200
622/622 [==============================] - 0s 71us/step - loss: 0.5116 - accuracy: 0.7878
Epoch 80/200
622/622 [==============================] - 0s 84us/step - loss: 0.5099 - accuracy: 0.7878
Epoch 81/200
622/622 [==============================] - 0s 75us/step - loss: 0.5139 - accuracy: 0.8006
Epoch 82/200
622/622 [==============================] - 0s 90us/step - loss: 0.5107 - accuracy: 0.7862
Epoch 83/200
622/622 [==============================] - 0s 75us/step - loss: 0.5078 - accuracy: 0.7878
Epoch 84/200
622/622 [==============================] - 0s 87us/step - loss: 0.5067 - accuracy: 0.7942
Epoch 85/200
622/622 [==============================] - 0s 80us/step - loss: 0.5082 - accuracy: 0.7846
Epoch 86/200
622/622 [==============================] - 0s 55us/step - loss: 0.5090 - accuracy: 0.8006
Epoch 87/200
622/622 [==============================] - 0s 51us/step - loss: 0.5039 - 

622/622 [==============================] - 0s 66us/step - loss: 0.4699 - accuracy: 0.8006
Epoch 158/200
622/622 [==============================] - 0s 95us/step - loss: 0.4682 - accuracy: 0.8119
Epoch 159/200
622/622 [==============================] - 0s 54us/step - loss: 0.4687 - accuracy: 0.8023
Epoch 160/200
622/622 [==============================] - 0s 91us/step - loss: 0.4680 - accuracy: 0.8006
Epoch 161/200
622/622 [==============================] - 0s 64us/step - loss: 0.4677 - accuracy: 0.8103
Epoch 162/200
622/622 [==============================] - 0s 75us/step - loss: 0.4678 - accuracy: 0.8071
Epoch 163/200
622/622 [==============================] - 0s 50us/step - loss: 0.4684 - accuracy: 0.8103
Epoch 164/200
622/622 [==============================] - 0s 49us/step - loss: 0.4678 - accuracy: 0.8103
Epoch 165/200
622/622 [==============================] - 0s 49us/step - loss: 0.4670 - accuracy: 0.8071
Epoch 166/200
622/622 [==============================] - 0s 54us/step - loss: 

In [30]:
score=classifier.evaluate(x_test,y_test,verbose=0)
score

[0.5063044520524832, 0.7836538553237915]